In [11]:
import os 
import time
import pandas as pd


import keras 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input

from keras import layers
from keras.layers import Conv2DTranspose
from tensorflow.keras import backend as K
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score


# Detection semi-supérvisé des anomalies des moteurs éléctriques

## Import des données

In [72]:
# train_data_2[['value','Class']].plot()

## Creation de modèle ( CNN AE ) 

In [33]:
class ConvAutoencoder:
    @staticmethod
    def build(width, height, depth, filters=(32, 64), latentDim=50):
        # initialize the input shape to be "channels last" along with
        # the channels dimension itself
        # channels dimension itself
        inputShape = (height, width, depth)
        chanDim = -1
        # define the input to the encoder
        inputs = Input(shape=inputShape)
        x = inputs
        # loop over the number of filters
        for f in filters:
        # apply a CONV => RELU => BN operation
            x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # flatten the network and then construct our latent vector
        volumeSize = K.int_shape(x)
        x = Flatten()(x)
        latent = Dense(latentDim)(x)
        # build the encoder model
        encoder = Model(inputs, latent, name="encoder")
        # start building the decoder model which will accept the
        # output of the encoder as its inputs
        latentInputs = Input(shape=(latentDim,))
        x = Dense(np.prod(volumeSize[1:]))(latentInputs)
        x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
        # loop over our number of filters again, but this time in
        # reverse order
        for f in filters[::-1]:
            # apply a CONV_TRANSPOSE => RELU => BN operation
            x = Conv2DTranspose(f, (3, 3), strides=2,
                padding="same")(x)
            x = LeakyReLU(alpha=0.2)(x)
            x = BatchNormalization(axis=chanDim)(x)
        # apply a single CONV_TRANSPOSE layer used to recover the
        # original depth of the image
        x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
        outputs = Activation("sigmoid")(x)
        # build the decoder model
        decoder = Model(latentInputs, outputs, name="decoder")
        # our autoencoder is the encoder + decoder
        autoencoder = Model(inputs, decoder(encoder(inputs)),
        name="autoencoder")
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
        autoencoder.compile(loss="mse", optimizer=optimizer)
        return autoencoder

## Preparation des données

'pure_1'

In [23]:
def build_spectrograms(input_folder,saving_folder):
    files = librosa.util.find_files(input_folder, ext=['wav']) 
    files = np.asarray(files)
    for y in files:  
        data = librosa.load(y, sr = 16000,mono = True)   
        data = data[0]     
        S_left = librosa.stft(data, center=True)
        fig, ax = plt.subplots()
        img = librosa.display.specshow(librosa.amplitude_to_db(S_left,
                                                               ref=np.max),
                                       y_axis='log', x_axis='time', ax=ax)
        ax.set_title('Power spectrogram')
        fig.colorbar(img, ax=ax, format="%+2.0f dB")
        file_name = y.split('/')[-1].split('.')[0]
        plt.savefig(f"{saving_folder}/{file_name}.png")
   

In [27]:
# load and transform sound to spectrogram
folders = [("./IDMT-ISA-ELECTRIC-ENGINE/train_cut/engine1_good","./Spectrograms/train_cut/engine1_good"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/train_cut/engine2_broken","./Spectrograms/train_cut/engine2_broken"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/train_cut/engine3_heavyload","./Spectrograms/train_cut/engine3_heavyload"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/test_cut/engine1_good","./Spectrograms/test_cut/engine1_good"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/test_cut/engine2_broken","./Spectrograms/test_cut/engine2_broken"),
           ("./IDMT-ISA-ELECTRIC-ENGINE/test_cut/engine3_heavyload","./Spectrograms/test_cut/engine3_heavyload")]

for input_folder,saving_folder in folders:
    build_spectrograms(input_folder,saving_folder)

In [27]:
# build_spectrograms(train_data_2,100,"Spectrograms6/Train",1)
# build_spectrograms(test_data_2,100,"Spectrograms6/Test",1)


## Chargement des spectrogrammes

In [37]:
#chargement custom des données de spéctrogramme de train
train_folder = "Spectrograms/train_cut/"
train_images = []
ano_images = [] 
class_data = ["engine1_good","engine2_broken","engine3_heavyload"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{train_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{train_folder}/{label}/{img}",color_mode = "grayscale",target_size=(200, 200))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label in ["engine1_good","engine3_heavyload"]:    
                train_images.append(input_arr)
            else:
                ano_images.append(input_arr)


In [39]:
X_train0_CAE = np.array(train_images) # spectrograms de données normal train
X_train1_CAE = np.array(ano_images) # spectrograms de données anormal train

In [38]:
#chargement custom des données de spéctrogramme de test
test_folder = "Spectrograms/test_cut"
test_images0 = []
test_images1 = [] 
class_data = ["engine1_good","engine2_broken","engine3_heavyload"]
for label in class_data:
    for i,img in enumerate(os.listdir(f"{test_folder}/{label}/")):
        if 'png' in img:
            image = tf.keras.utils.load_img(f"{test_folder}/{label}/{img}",color_mode = "grayscale",target_size=(200, 200))
            input_arr = tf.keras.utils.img_to_array(image)/255.0
            if label in ["engine1_good","engine3_heavyload"]:    
                test_images0.append(input_arr)
            else:
                test_images1.append(input_arr)

In [40]:
X_test0CAE = np.array(test_images0) # spectrograms de données normal test
X_test1CAE = np.array(test_images1) # spectrograms de données anormal test

## Apprentissage du modèle

In [35]:
for e in X_test0CAE:
    if e.shape != (255, 255, 1):
        print('ow')

In [41]:
CAE = ConvAutoencoder.build(200, 200, 1) # construction de l'architecture

In [43]:
start = time.time()
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
CAE.fit(X_train0_CAE,X_train0_CAE, epochs=1000,batch_size = 10, verbose = True, callbacks= [callback])
print(f"model trained in {(time.time()-start)/60} minutes")


Epoch 1/1000
24/24 [==============================] - 8s 323ms/step - loss: 0.0131
Epoch 2/1000
24/24 [==============================] - 9s 361ms/step - loss: 0.0077
Epoch 3/1000
24/24 [==============================] - 8s 353ms/step - loss: 0.0058
Epoch 4/1000
24/24 [==============================] - 9s 362ms/step - loss: 0.0043
Epoch 5/1000
24/24 [==============================] - 9s 367ms/step - loss: 0.0039
Epoch 6/1000
24/24 [==============================] - 8s 352ms/step - loss: 0.0042
Epoch 7/1000
24/24 [==============================] - 8s 346ms/step - loss: 0.0034
Epoch 8/1000
24/24 [==============================] - 9s 360ms/step - loss: 0.0031
Epoch 9/1000
24/24 [==============================] - 9s 368ms/step - loss: 0.0027
Epoch 10/1000
24/24 [==============================] - 11s 460ms/step - loss: 0.0025
Epoch 11/1000
24/24 [==============================] - 12s 484ms/step - loss: 0.0021
Epoch 12/1000
24/24 [==============================] - 10s 424ms/step - loss: 0.002

## Prediction

In [44]:
predictions0CAE = CAE.predict(X_train0_CAE) # normal data predictions
predictions1CAE = CAE.predict(X_train1_CAE) # anomalies predictions

In [48]:
r_e_CAE = np.sum(np.sum((X_train0_CAE-predictions0CAE) ** 2, axis=1),axis = 1) # computing reconstruction error of normal train data 
r_e_anos_CAE = np.sum(np.sum((X_train1_CAE-predictions1CAE) ** 2, axis=1),axis = 1) # computing reconstruction error of abnormal train data 
threshold_t_CAE = np.quantile(r_e_CAE,0.9) # setting reconstruction error threshold

In [49]:
prediction_test0CAE = CAE.predict(X_test0CAE)
prediction_test1CAE = CAE.predict(X_test1CAE)

In [50]:
r_e_test0CAE = np.sum(np.sum((X_test0CAE-prediction_test0CAE) ** 2, axis=1),axis=1)
r_e_test1CAE = np.sum(np.sum((X_test1CAE-prediction_test1CAE) ** 2, axis=1),axis=1)

In [51]:
decision0CAE = -r_e_test0CAE + threshold_t_CAE
decision1CAE = -r_e_test1CAE + threshold_t_CAE
df0CAE = pd.DataFrame(decision0CAE, columns=['v'])
df1CAE = pd.DataFrame(decision1CAE, columns=['v'])

In [52]:
# computing f1 score of minority class
recall = len(df1CAE[df1CAE.v<0])/len(df1CAE)
precision = len(df1CAE[df1CAE.v<0])/(len(df1CAE[df1CAE.v<0])+len(df0CAE[df0CAE.v<0]))
f1 = 2*precision*recall/(precision+recall)

In [53]:
# computing f1 score of majority class
recall_maj = len(df0CAE[df0CAE.v>0])/len(df0CAE)
precision_maj = len(df0CAE[df0CAE.v>0])/(len(df1CAE[df1CAE.v>0])+len(df0CAE[df0CAE.v>0]))
f1_maj = 2*precision_maj*recall_maj/(precision_maj+recall_maj)

ZeroDivisionError: division by zero

### Test score

In [ ]:
print(f"Test scores:")
print(f"minority class F1 score {f1}")
print(f"majority class F1 score {f1_maj}")
print(f"average F1 score {(f1+f1_maj)/2}")